In [1]:
from Game import Game
from Deck import Deck, Card
from players.Player import Player
from players.HumanPlayer import HumanPlayer
from players.RandomPlayer import RandomPlayer
from PlayerNode import PlayerNode, link_players, remove_player, goto_player
from colorama import init as colorama_init
from colorama import Fore
from colorama import Style
import numpy as np
import math

# Analysis of game statistics (given random players)

the question to anser are:
- Who wins the most?
- Who loses the most?
- Is there a best strategy for predictions?
- Is there a best strategy for playing cards?
- Are the first or last player of a round in advantage?
- What is the right number to predict with n player and c cards?

Check if there is a connection between winning and starting first or last

In [ ]:
class FixedGame(Game):
    "A game where players never change position, 0 always first, 4 always last"

    def play(self, verbose=True, show_hands=True):
        for round in range(self.max_rounds):
            # hand_size = 5 - (round % 5)
            hand_size = 5 - ((round+self.starting_round) % 5)
            
            if verbose:
                print(f"\n--- Round {round+1}: {hand_size} card(s) ---")
            
            # deal hand
            for player in self.players:
                hand = self.deck.draw_hand(hand_size)
                player.take_hand(hand)
                card_strs = hand[0].strc()
                for card in hand[1:]:
                    card_strs += ", " + card.strc()
                if verbose and show_hands:
                    print(f"Player {player.id} hand: [{card_strs}]")

            # make prediction
            current_preds = []
            player_node = self.current_player
            for _ in range(len(self.players)):
                if player_node.next.player.id == self.current_player.player.id:
                    last = True
                else:
                    last = False
                p = player_node.player.make_prediction(current_preds, last, hand_size)
                current_preds.append(p)
                self.predictions[round, player_node.player.id] = p
                if verbose:
                    print(f"Player {player_node.player.id} prediction: {p}")
                player_node = player_node.next
                # TODO check last prediction
            player_node = self.current_player

            # play cards
            for turn in range(hand_size):
                self.first_in_turn[round, player_node.player.id] += 1
                self.last_in_turn[round, goto_player(player_node, (player_node.player.id+len(self.players)-1)%len(self.players)).player.id] += 1
                if verbose:
                    print(f"--- Turn {turn+1} ---")
                played_cards = {}
                for _ in range(len(self.players)):
                    card = player_node.player.play_card(list(map(lambda c: str(c), played_cards.values())))
                    if verbose:
                        print(f"Player {player_node.player.id} plays: {card.strc()}")
                    played_cards[player_node.player.id] = card
                    player_node = player_node.next

                if not len(played_cards.values()) == len(self.players):
                    raise ValueError("Not all players played a card")
                
                # determine catches
                ace = False
                for id, card in played_cards.items():
                    if card.rank == '1' and card.suit == 'Denari':
                        if self.predictions[round, id] > self.catches[round, id]:
                            ace = True
                            self.catches[round, id] += 1
                            # player_node = goto_player(player_node, id)
                            if verbose:
                                print(f"Player {id} takes")
                        else:
                            played_cards[id] = Card('Bastoni', '1')  # lowest possible card

                if not ace:
                    max = None
                    max_idx = None
                    for idx, card in played_cards.items():
                        if max is None or card > max:
                            max = card
                            max_idx = idx
                    self.catches[round, max_idx] += 1
                    # player_node = goto_player(player_node, max_idx)
                    if verbose:
                        print(f"Player {max_idx} takes")

            if verbose:
                print(" ")

            # check errors
            removed_players = []
            for i, player in enumerate(self.players):
                e = abs(self.predictions[round, i] - self.catches[round, i])
                if e > 0:
                    self.errors[round, i] = e
                    if verbose:
                        print(f"{Fore.MAGENTA}Player {player.id} made {int(e)} error(s){Style.RESET_ALL}")
                if np.sum(self.errors[:, player.id]) > self.max_errors:
                    # delete player
                    if verbose:
                        print(f"Player {player.id} is eliminated.")
                    p = self.players.pop(i)
                    remove_player(self.current_player, p.id)
                    removed_players.append(p.id)
            
            if removed_players:
                if len(self.players) < 1:
                    print("Game Over")
                    print("Draw")
                    return
                if len(self.players) == 1:
                    print("Game Over")
                    print(f"Winner: {self.players[0].id}")
                    return
            self.current_player = link_players(self.players)
            self.deck = Deck()

        if verbose:
            print("Game Over")
            print(f"Players remaining: {[player.id for player in self.players]}")



In [8]:
reps = 10000

stat_first = np.zeros((reps, 5))
stat_last = np.zeros((reps, 5))
stat_winners = np.zeros((reps, 5))

for t in range(reps):
    game = FixedGame(n=5, r=1, e=-1, s=4)
    game.play(verbose=False)
    stats = game.return_stats()
    stat_first[t, :] = stats["tot_first"]
    stat_last[t, :] = stats["tot_last"]
    stat_winners[t, :] = stats["winners"]


for i in range(5):
    first = np.sum((stat_first[:, i]))
    last = np.sum((stat_last[:, i]))
    wins = np.sum(stat_winners[:, i] == 1)
    losses = np.sum(stat_winners[:, i] == 0)
    print(f"Player {i}: First to play: {first} times, Last to play: {last} times, Total wins: {wins}, Total losses: {losses}")


Player 0: First to play: 10000.0 times, Last to play: 0.0 times, Total wins: 5492, Total losses: 4508
Player 1: First to play: 0.0 times, Last to play: 0.0 times, Total wins: 5375, Total losses: 4625
Player 2: First to play: 0.0 times, Last to play: 0.0 times, Total wins: 5509, Total losses: 4491
Player 3: First to play: 0.0 times, Last to play: 0.0 times, Total wins: 5255, Total losses: 4745
Player 4: First to play: 0.0 times, Last to play: 10000.0 times, Total wins: 4852, Total losses: 5148
